In [ ]:
# HiC-Pro 
def preparing_HiCPro_file(h3dgipdir, genome, enzyme, input_dir):
    import os
    output_file = "./HiCPro/run_HiC-Pro.sh"
    fp = open(output_file, 'w')
    # 1、Enzyme fragments
    enzyme_fragments_order="python {0}/utils/digest_genome.py {1} -r {2} -o HiC-Pro_enzyme_fragments.bed".format(h3dgipdir, genome, enzyme)
    print(enzyme_fragments_order, file=fp)
    # 2、Chromosome size
    chromosome_size_step1 = "samtools faidx {0} -o {0}.fai".format(genome)
    chromosome_size_step2 = "awk -v OFS='\\t' " + " \'{print $1, $2}\' " + "{0}.fai > HiC-Pro_chromosome_size.txt".format(genome)
    print(chromosome_size_step1, file=fp)
    print(chromosome_size_step2, file=fp)
    # 3、genome index
    ln_order = "ln -s {}".format(genome)
    genomeName = genome.split("/")[-1].split(".")[0]
    build_index_order = "bowtie2-build -f {0} --threads 4".format(genomeName)
    print(ln_order, file=fp)
    print(build_index_order, file=fp)
    # 4、change configure file
    cp_order = "cp {}/utils/HiC-Pro.config .".format(h3dgipdir)
    tmpDir = os.getcwd()
    workDir = tmpDir.replace("/", "\/")
    HiCProDir = "{}\/HiCPro".format(workDir)
    sed_order1 = "sed -i 's/InderDir/{0}/g' HiC-Pro.config".format(HiCProDir)
    sed_order2 = "sed -i 's/GenomeName/{0}/g' HiC-Pro.config".format(genomeName)
    sed_order3 = "sed -i 's/ChromosomeSize/{0}\/HiC-Pro_chromosome_size.txt/g' HiC-Pro.config".format(HiCProDir)
    ## HindIII
    sed_order4 = "sed -i 's/EnzymeFragment/{0}\/HiC-Pro_enzyme_fragments.bed/g' HiC-Pro.config".format(HiCProDir)
    print(sed_order1, file=fp)
    print(sed_order2, file=fp)
    print(sed_order3, file=fp)
    print(sed_order4, file=fp)
    # 4 run HiC-Pro
    HiCPro_step1 = "HiC-Pro -c {0}/utils/HiC-Pro.config -i {1} -o HiCPro_result -p".format(h3dgipdir, input_dir)
    HiCPro_step2 = "cd HiCPro_result"
    HiCPro_step3 = "sh HiCPro_step1.sh"
    HiCPro_step4 = "sh HiCPro_step2.sh"
    HiCPro_step5 = "cd .."
    print(HiCPro_step1, file=fp)
    print(HiCPro_step2, file=fp)
    print(HiCPro_step3, file=fp)
    print(HiCPro_step4, file=fp)
    print(HiCPro_step5, file=fp)
    fp.close()

# Juicer    
def preparing_juicer_file(h3dgipdir, genome, enzyme, input_dir, compartment_resolution, tad_resolution, loop_resolution, name):
    import os
    output_file = "juicer/run_juicer.sh"
    fp = open(output_file, "w")
    genomeName = genome.split("/")[-1].split(".")[0]
    # 1、Enzyme fragm
    enzyme_fragments_order='python {0}/utils/generate_site_positions.py {1} juicer_enzyme_fragments {2}'.format(h3dgipdir, enzyme, genome)
    print(enzyme_fragments_order, file=fp)
    # 2、Chromosome size
    chromosome_size_step1 = "samtools faidx {0} -o {0}.fai".format(genome)
    chromosome_size_step2 = "awk -v OFS='\\t'" + " \'{print $1, $2}\' " + "{0}.fai > juicer_chromosome_size.txt".format(genome)
    print(chromosome_size_step1, file=fp)
    print(chromosome_size_step2, file=fp)    
    # 3、genome index
    ln_order = "ln -s {0} juicer/{0}".format(genome)
    build_index_order = "bwa index {0}".format(genome)
    print(ln_order, file=fp)
    print(build_index_order, file=fp)
    # 4、Generates .hic file
    workDir = os.getcwd()
    genome_name = genome.split("/")[-1].split(".")[0]
    juicer_order = "juicer.sh -z {0} -y {1}/juicer/juicer_enzyme_fragments_{2}.txt -p {1}/juicer/juicer_chromsome_size.txt -d {3} -D juicer_output/ -t 4".format(genome, workDir, enzyme, input_dir)
    print(juicer_order, file=fp)
    # 5、divide Compartment
    juicer_compartment = "java -jar {0}/utils/juicer_scripts/juicer_tools.jar eigenvector KR juicer_output/{1}.hic chr1 BP {2}".format(h3dgipdir, name, compartment_resolution)
    print(juicer_compartment, file=fp)
    print("#divide chr1 compartment", file=fp)
    # 6、Identify TADs
    juicer_TAD = "java -jar {0}/utils/juicer_scripts/juicer_tools.jar arrowhead -m 2000 -r {1} -k KR --threads 4 juicer_output/{2}.hic juicer_output".format(h3dgipdir, tad_resolution, name)
    print(juicer_TAD, file=fp)
    # 7、Infer loops
    juicer_loop = "java -jar {0}/utils/juicer_scripts/juicer_tools.jar hiccups --cpu -r {1} -f 0.1 -p 4 -i 7 -d 20000 -t 0.02,1.5,1.75,2 -k KR --threads 4  juicer_output/{2}.hic juicer_output".format(h3dgipdir, loop_resolution, name)
    print(juicer_loop, file=fp)
    fp.close()
    
# TADLib
def preparing_TADLib_file(h3dgipdir, filename, resolution):
    matrix = '../HiCPro_result/hic_results/matrix/raw/{0}/{1}_{0}.matrix'.format(resolution, filename)
    bins = '../HiCPro_result/hic_results/matrix/raw/{0}/{1}_{0}_abs.bed'.format(resolution, filename)
    output_file = 'TADLib/run_TADLib.sh'
    fp = open(output_file, "w")
    # 1、split chr
    split_chr_order = "python {0}/utils/hicproTocool.py -m {1} -b {2} -o TADLib/intramtx -YN N".format(h3dgipdir, matrix, bins)
    print(split_chr_order, file=fp)
    # 2、to cool
    dataset_file = 'TADLib/dataset'
    fd = open(dataset_file, 'w')
    print("res:{}".format(resolution), file=fd)
    print(" ./intramtx", file=fd)
    fd.close()
    tocool_order = "toCooler -O TADLib/TADLib.cool -d dataset --chromsize-file ../HiCPro/HiC-Pro_chromosome_size.txt"
    print(tocool_order, file=fp)
    # 3、find tad
    meta_file = 'TADLib/meta_file'
    fm = open(meta_file, 'w')
    print("res:{}".format(resolution), file=fm)
    print(" rep1:TADLib/TADLib.cool::{}".format(resolution), file=fm)
    fm.close()
    find_TAD_order = "hitad -O TADLib_result/TADLib_TAD.bed -d meta_file --logFile hitad.log -p 4"
    print(find_TAD_order, file=fp)
    fp.close()

# FitHiC 
def preparing_FitHiC_file(h3dgipdir, filename, resolution):
    matrix = '../HiCPro_result/hic_results/matrix/raw/{0}/{1}_{0}.matrix'.format(resolution, filename)
    bins = '../HiCPro_result/hic_results/matrix/raw/{0}/{1}_{0}_abs.bed'.format(resolution, filename)
    output_file = 'FitHiC/run_FitHiC.sh'
    fp = open(output_file, "w")
    fithic_step1 = "python {0}/utils/hicpro2fithic.py -i {1} -b {2} -r {3} -o ./ -n FitHiC".format(h3dgipdir, matrix, bins, resolution)
    fithic_step2 = "fithic -f ./FitHiC.fithic.fragmentMappability.gz -i ./FitHiC.fithic.interactionCounts.gz -r {} -L 6000 -U 3000000 -p 2 -o FitHiC_result -l FitHiC".format(resolution) # 3 Kb resolution
    print(fithic_step1, file=fp)
    print(fithic_step2, file=fp)
    fp.close()

# hichipper
def preparing_hichipper_file():
    # build the config file of hichipper
    import os
    workDir = os.getcwd() 
    fy = open('./hichipper/hichipper_config.yaml', 'w')
    print("peaks:", file=fy)
    print("  - COMBINED,ALL", file=fy)
    print("resfrags:", file=fy)
    print("  - {0}/HiCPro/HiC-Pro_enzyme_fragments.bed".format(workDir), file=fy)
    print("hicpro_output:", file=fy)
    print("  - {0}/HiCPro_result/hic_reuslts".format(workDir), file=fy)
    fy.close()
    # build the run file of hichipper
    output_file = "./hichipper/run_hichipper.sh"
    fp = open(output_file, "w")
    hichipper_order = "hichipper --out hichipper_result hichipper_config.yaml"
    print(hichipper_order, file=fp)
    fp.close()

# FitHiChIP
def preparing_FitHiChIP_file(h3dgipdir, name):
    output_file = "./FitHiChIP/run_FitHiChIP.sh"
    fp = open(output_file, "w")
    # configure file
    cp_order = "cp {}/utils/configfile_BiasCorrection_ICEBias ./".format(h3dgipdir)
    sed_order1 = "sed -i 's/ValidPairs/..\/HiCPro\/hic_results\/data\/{0}\/{0}.allValidPairs/g' configfile_BiasCorrection_ICEBias".format(name)
    sed_order2 = "sed -i 's/chromSize/..\/HiCPro\/HiC-Pro_chromosome_size.txt/g' configfile_BiasCorrection_ICEBias"
    print(cp_order, file=fp)
    print(sed_order1, file=fp)
    print(sed_order2, file=fp)
    FitHiChIP_order = "bash {0}/utils/FitHiChIP_HiCPro.sh -C configfile_BiasCorrection_ICEBias".format(h3dgipdir)
    print(FitHiChIP_order, file=fp)
    print("# The optional compilation file contains configfile_BiasCorrection_ICEBias, configfile_BiasCorrection_CoverageBias, configfile_P2P_BiasCorrection_CoverageBias, configfile_P2P_BiasCorrection_ICEBias", file=fp)
    fp.close()

# creat directory
def preparing_directory():    
    def make_dir(dirname):
        import os
        work_path = os.getcwd()
        file_path = "{0}/{1}".format(work_path, dirname)
        print(file_path)
        if os.path.exists(file_path):
            print('The folder exists')
        else:
            os.makedirs(file_path)
    all_dir = ['HiCPro', 'TADLib', 'FitHiC', 'hichipper', 'FitHiChIP', 'juicer']
    for d in all_dir:
        make_dir(d)


# build workflows
def H3DGIP_workflows(h3dgipdir, input_dir, genome, enzyme, compartment_resolution, tad_resolution, loop_resolution, name):
    #make directory
    preparing_directory()
    # HiC-Pro
    preparing_HiCPro_file(h3dgipdir, genome, enzyme, input_dir)
    # Juicer
    preparing_juicer_file(h3dgipdir, genome, enzyme, input_dir, compartment_resolution, tad_resolution, loop_resolution, name)
    # TADLib
    preparing_TADLib_file(h3dgipdir, name, tad_resolution)
    # FitHiC 
    preparing_FitHiC_file(h3dgipdir, name, loop_resolution)
    # hichipper
    preparing_hichipper_file()
    # FitHiChIP
    preparing_FitHiChIP_file(h3dgipdir, name)
    # write the pipeline
    fp = open("H3DGIP_pipeline.sh", 'w')
    for i in ['HiCPro', 'juicer', 'TADLib', 'FitHiC', 'FitHiChIP', 'hichipper']:
        print("#{}".format(i), file=fp)
        print("cd {}".format(i), file=fp)
        print("sh run_{}.sh".format(i), file=fp)
        print("cd ..", file=fp)
        print("", file=fp)
    fp.close()

def main(argv):
    import sys
    import argparse
    parser = argparse.ArgumentParser(description="")
    parser.add_argument("-d", "--h3dgip_directory", required=True, help="The directory of H3DGIP")
    parser.add_argument('-i', '--input_directory', required=True, help="The directory of input file")
    parser.add_argument('-n', '--sample_name', required=True, help="Sample name")
    parser.add_argument('-g', '--genome', required=True, help="The genome file")
    parser.add_argument('-e', '--enzyme', required=True, help="The type of restriction enzyme")
    parser.add_argument('-c', '--compartment_resolution', required=True, help="The resolution of compartment")
    parser.add_argument('-t', '--tad_resolution', required=True, help='The resolution of TAD')
    parser.add_argument('-l', '--loop_resolution', required=True, help='The resolution of loop')
    args = parser.parse_args()
    h3dgipdir, input_dir, name, genome, enzyme, compartment_resolution, tad_resolution, loop_resolution  = args.h3dgip_directory, args.input_directory, args.sample_name, args.genome, args.enzyme, args.compartment_resolution, args.tad_resolution, args.loop_resolution
    H3DGIP_workflows(h3dgipdir, input_dir, genome, enzyme, compartment_resolution, tad_resolution, loop_resolution, name)
import sys
if __name__ == "__main__":
    main(sys.argv[1:])

In [ ]:
import sys
import os
import pandas as pd
import argparse
def main(argv):
    parser = argparse.ArgumentParser(description="")
    parser.add_argument('-b', "--bedfile", required=True, help="The bed file generated by HiC-Pro")
    parser.add_argument('-m', '--matrixfile', required=True, help="Matrix file generated by HiC-Pro")
    parser.add_argument('-o', "--outdir", required=True, help = 'Output file directory')
    parser.add_argument('-YN', '--Scaffold', required=True, help="Is scaffold internal interaction required")
    args = parser.parse_args()
    bed_f = args.bedfile
    mtx_f = args.matrixfile
    out_dir = args.outdir
    is_scaf = args.Scaffold
    df_bed = pd.read_csv(bed_f, sep='\t', names=['chr', 's', 'e', 'id'])
    df_mtx = pd.read_csv(mtx_f, sep='\t', names=['bin1', 'bin2', 'reads'])
    #Determine if the scaffold id need to be remove
    if is_scaf == 'NO' or is_scaf == 'no' or is_scaf == 'N':
        df_bed = df_bed.query("~chr.str.contains('Scaff')", engine='python').copy()
        #df_bed = df_bed.query("~chr.str.contains('Contig')", engine='python').copy()
    # get need chr id
    chr_id = sorted(set(list(df_bed['chr'])))
    #print(chr_id)
    n = 1
    for c in chr_id:
        names = "{0}_{0}.txt".format(n)
        out_f1 = os.path.join(out_dir, names)
        out_f2 = os.path.join(out_dir, c+"_abs.bed")
        df_chr = df_bed.query("chr == @c").copy()
        id_start = df_chr['id'].min()
        id_end = df_chr['id'].max()
        df_chr_mtx = df_mtx.query("(bin1 >= @id_start) & (bin1 <= @id_end) & (bin2 >= @id_start) & (bin2 <= @id_end)").copy()
        df_chr_mtx['bin1'] = df_chr_mtx['bin1'] - id_start + 1
        df_chr_mtx['bin2'] = df_chr_mtx['bin2'] - id_start + 1
        df_chr['id'] = df_chr['id'] - id_start + 1
        df_chr_mtx.to_csv(out_f1, sep='\t', index=False, header=False)
        df_chr.to_csv(out_f2, sep='\t', index=False, header=False)
        n += 1
if __name__ == "__main__":
    main(sys.argv[1:])

In [ ]:
import argparse
import re
import os
import sys
import numpy as np

RE_cutsite = {
    "mboi": ["^GATC"],
    "dpnii": ["^GATC"],
    "bglii": ["A^GATCT"],
    "hindiii": ["A^AGCTT"]}


def find_re_sites(filename, sequences, offset):
    with open(filename, 'r') as infile:
        chr_id = None
        big_str = ""
        indices = []
        all_indices = []
        contig_names = []
        c = 0
        for line in infile:
            c += 1
            if line.startswith(">"):
                print("{}...".format(line.split()[0][1:]))
                # If this is not the first chromosome, find the indices and append
                # them to the list
                if chr_id is not None:
                     for rs in range(len(sequences)):
                         pattern = "(?={})".format(sequences[rs].lower())
                         indices += [m.start() + offset[rs]\
                         for m in re.finditer(pattern, big_str)]
                     indices.sort()
                     all_indices.append(indices)
                     indices = []

                # This is a new chromosome. Empty the sequence string, and add the
                # correct chrom id
                big_str = ""
                chr_id = line.split()[0][1:]
                if chr_id in contig_names:
                    print("The fasta file contains several instance of {}. Exit.".format(chr_id))
                    sys.exit(-1)
                contig_names.append(chr_id)
            else:
                # As long as we don't change chromosomes, continue reading the
                # file, and appending the sequences
                big_str += line.lower().strip()
        # Add the indices for the last chromosome
        for rs in range(len(sequences)):
            pattern = "(?={})".format(sequences[rs].lower())
            indices += [m.start() + offset[rs]
                        for m in re.finditer(pattern, big_str)]
        indices.sort()
        all_indices.append(indices)
    
    return contig_names, all_indices


def find_chromsomose_lengths(reference_filename):
    chromosome_lengths = []
    chromosome_names = []
    length = None
    with open(reference_filename, 'r') as infile:
        for line in infile:
            if line.startswith(">"):
                chromosome_names.append(line[1:].strip())
                if length is not None:
                    chromosome_lengths.append(length)
                length = 0
            else:
                length += len(line.strip())
        chromosome_lengths.append(length)
    return chromosome_names, np.array(chromosome_lengths)


def replaceN(cs):
    npos = int(cs.find('N'))
    cseql = []
    if npos != -1:
        for nuc in ["A","C","G","T"]:
            tmp = cs.replace('N', nuc, 1)
            tmpl = replaceN(tmp)
            if type(tmpl) == list:
                cseql = cseql + tmpl
            else:
                cseql.append(tmpl)
    else:
        cseql.append(cs)
    return cseql


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument('fastafile')
    parser.add_argument('-r', '--restriction_sites',
                        dest='res_sites',
                        nargs='+',
                        help=("The cutting position has to be specified using "
                              "'^'. For instance, -r A^AGCTT for HindIII "
                              "digestion. Several restriction enzyme can be "
                              "specified."))
    parser.add_argument('-o', '--out', default=None)
    args = parser.parse_args()

    filename = args.fastafile
    out = args.out
    
    # Split restriction sites if comma-separated
    cutsites=[]
    for s in args.res_sites:
        for m in s.split(','):
            cutsites.append(m)
                
    # process args and get restriction enzyme sequences
    sequences = []
    offset = []
    for cs in cutsites:
        if cs.lower() in RE_cutsite:
            cseq = ''.join(RE_cutsite[cs.lower()])
        else:
            cseq = cs

        offpos = int(cseq.find('^'))
        if offpos == -1:
            print("Unable to detect offset for {}. Please, use '^' to specify the cutting position,\
                   i.e A^GATCT for HindIII digestion.".format(cseq))
            sys.exit(-1)

        for nuc in list(set(cseq)):
            if nuc not in ['A','T','G','C','N','^']:
                print("Find unexpected character ['{}']in restriction motif".format(nuc))
                print("Note that multiple motifs should be separated by a space (not a comma !)")

                sys.exit(-1)

        offset.append(offpos)
        sequences.append(re.sub('\^', '', cseq))

    # replace all N in restriction motif
    sequences_without_N = []
    offset_without_N = []
    for rs in range(len(sequences)):
        nrs = replaceN(sequences[rs])
        sequences_without_N = sequences_without_N + nrs
        offset_without_N = offset_without_N + [offset[rs]] * len(nrs)
          
    sequences = sequences_without_N
    offset = offset_without_N
    
    if out is None:
        out = os.path.splitext(filename)[0] + "_fragments.bed"

    print("Analyzing", filename)
    print("Restriction site(s)", ",".join(sequences))
    print("Offset(s)",  ','.join(str(x) for x in offset))

    # Read fasta file and look for rs per chromosome
    contig_names, all_indices = find_re_sites(filename, sequences,  offset=offset)
    _, lengths = find_chromsomose_lengths(filename)

    valid_fragments = []
    for i, indices in enumerate(all_indices):
        valid_fragments_chr = np.concatenate(
            [np.concatenate([[0], indices])[:, np.newaxis],
             np.concatenate([indices, [lengths[i]]])[:, np.newaxis]],
            axis=1)
        valid_fragments.append(valid_fragments_chr)

    # Write results
    print("Writing to {} ...".format(out))
    with open(out, 'w') as outfile:
        for chrom_name, indices in zip(contig_names, valid_fragments):
            frag_id = 0
            for begin, end in indices:
                # allow to remove cases where the enzyme cut at
                # the first position of the chromosome
                if end > begin:
                    frag_id += 1
                    frag_name = "HIC_{}_{}".format(str(chrom_name), int(frag_id))
                    outfile.write("{}\t{}\t{}\t{}\t0\t+\n".format(str(chrom_name), int(begin), int(end), str(frag_name)))